# Introduction to AI Chat with Ollama! 🤖

Welcome to this fun tutorial where we'll learn how to chat with AI models using Python! We'll be using something called **Ollama**, which lets us run AI models right on our own computers.

In this tutorial, we'll learn:
1. How to have a simple chat with an AI
2. How to make the AI give us specific types of answers (like Yes/No or Multiple Choice)
3. How to save our conversations with the AI

Let's get started! 🚀

## Setup 🛠️

First, we need to make sure we have all the tools we need. We'll use a few Python packages:
- `ollama`: To talk to our AI models
- `pydantic`: To help us get structured responses from the AI

Let's install these packages if we don't have them already:


## Part 1: Simple Chat with AI 💬

Let's start by importing the packages we need and creating a simple function to chat with our AI. We'll make it super easy to use!


In [ ]:
import os
from datetime import datetime
from ollama import chat

def chat_with_ai(prompt: str, model: str = "llama2", temperature: float = 0.7):
    """
    Have a simple chat with an AI model!
    
    Args:
        prompt: Your question or message to the AI
        model: Which AI model to use (default is llama2)
        temperature: How creative the AI should be (0.0 = very focused, 1.0 = very creative)
    
    Returns:
        The AI's response
    """
    try:
        # Send our message to the AI
        response = chat(
            model=model,
            options={"temperature": temperature},
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Get the AI's response
        return response["message"]["content"]
    
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

# Let's test our function with a simple question!
response = chat_with_ai("What is artificial intelligence? Explain it to a high school student.")
print("AI's response:\n", response)

### Saving Our Conversations 📝

It's often useful to save our conversations with the AI for later. Let's create a function that helps us do that!


In [14]:
def save_chat(model: str, prompt: str, response: str):
    """
    Save our chat to a file so we can look at it later!
    
    Args:
        model: The AI model we used
        prompt: What we asked the AI
        response: What the AI answered
    """
    # Create a folder to store our chats if it doesn't exist
    results_dir = os.path.join("results", "chats")
    os.makedirs(results_dir, exist_ok=True)
    
    # Create a filename with the current time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(results_dir, f"chat_{timestamp}.txt")
    
    # Save the conversation
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Model: {model}\n")
        f.write(f"\nOur Question:\n{prompt}\n")
        f.write(f"\nAI's Answer:\n{response}\n")
    
    print(f"\nChat saved to: {filename}")

# Let's try asking another question and save it!
prompt = "What are three fun projects a high school student could do to learn about AI?"
response = chat_with_ai(prompt)
print("AI's response:\n", response)
save_chat("llama2", prompt, response)


AI's response:
 Oops! Something went wrong: model "llama2" not found, try pulling it first (status code: 404)

Chat saved to: results/chats/chat_20250622_114409.txt


## Part 2: Getting Specific Answers from AI 🎯

Sometimes we want the AI to give us very specific types of answers, like:
- Just "Yes" or "No"
- True or False
- Multiple choice (A, B, C, or D)

This is called "structured output" because we're telling the AI exactly how to structure its answer. Let's see how to do this!


In [15]:
from pydantic import BaseModel
from typing import Literal

# These classes tell the AI exactly what kind of answer we want
class YesNoAnswer(BaseModel):
    """For questions that should be answered with Yes or No"""
    answer: Literal["Yes", "No"]

class TrueFalseAnswer(BaseModel):
    """For statements that should be marked True or False"""
    answer: bool

class MultipleChoiceAnswer(BaseModel):
    """For questions with A, B, C, or D options"""
    answer: Literal["A", "B", "C", "D"]

def get_structured_answer(prompt: str, answer_type: BaseModel, model: str = "llama2"):
    """
    Get a specific type of answer from the AI
    
    Args:
        prompt: Your question
        answer_type: What kind of answer you want (YesNoAnswer, TrueFalseAnswer, or MultipleChoiceAnswer)
        model: Which AI model to use
    """
    # Tell the AI what kind of answer we want
    system_prompt = f"""
    You must respond in valid JSON format.
    Your answer must follow this structure:
    {answer_type.model_json_schema()}
    """
    
    try:
        # Ask the AI
        response = chat(
            model=model,
            options={"temperature": 0.0},  # We want consistent answers here
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            format="json"
        )
        
        # Get and validate the answer
        content = response["message"]["content"]
        validated_answer = answer_type.model_validate_json(content)
        return validated_answer
        
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

# Let's try some examples!

# Yes/No question
yes_no_q = "Is Python a programming language?"
yes_no_answer = get_structured_answer(yes_no_q, YesNoAnswer)
print(f"\nQuestion: {yes_no_q}")
print(f"Answer: {yes_no_answer.answer if hasattr(yes_no_answer, 'answer') else yes_no_answer}")

# True/False statement
true_false_q = "The Earth is flat."
true_false_answer = get_structured_answer(true_false_q, TrueFalseAnswer)
print(f"\nStatement: {true_false_q}")
print(f"Answer: {true_false_answer.answer if hasattr(true_false_answer, 'answer') else true_false_answer}")

# Multiple choice question
multiple_choice_q = """
What is the main purpose of a computer's CPU?
A) Store data permanently
B) Process instructions and perform calculations
C) Display images on the screen
D) Connect to the internet
"""
mc_answer = get_structured_answer(multiple_choice_q, MultipleChoiceAnswer)
print(f"\nQuestion: {multiple_choice_q}")
print(f"Answer: {mc_answer.answer if hasattr(mc_answer, 'answer') else mc_answer}")



Question: Is Python a programming language?
Answer: Oops! Something went wrong: model "llama2" not found, try pulling it first (status code: 404)

Statement: The Earth is flat.
Answer: Oops! Something went wrong: model "llama2" not found, try pulling it first (status code: 404)

Question: 
What is the main purpose of a computer's CPU?
A) Store data permanently
B) Process instructions and perform calculations
C) Display images on the screen
D) Connect to the internet

Answer: Oops! Something went wrong: model "llama2" not found, try pulling it first (status code: 404)


## Chat with Memory: Having Real Conversations! 🧠

One of the coolest things about chatting with AI is that it can remember what you talked about earlier in the conversation. This is called "chat history" or "context".

For example, if you ask about the weather and then ask "Why is that?", the AI will know you're asking about the weather!

Let's try having a conversation where the AI remembers what we talked about:


In [ ]:
from ollama import chat

# This is our conversation history - it's like the AI's memory!
messages = [
    # First, we ask about the sky
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },
    # The AI's response about the sky
    {
        'role': 'assistant',
        'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
    },
    # Then we ask about weather
    {
        'role': 'user',
        'content': 'What is the weather in Tokyo?',
    },
    # The AI's response about Tokyo weather
    {
        'role': 'assistant',
        'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing.',
    },
]

def display_chat_history():
    """Show the conversation so far"""
    print("\n🗨️ Chat History:")
    for msg in messages:
        if msg['role'] == 'user':
            print(f"\n👤 You: {msg['content']}")
        else:
            print(f"🤖 AI: {msg['content']}")
    print("\n" + "="*50 + "\n")

# Show our chat history so far
display_chat_history()

# Let's have a conversation!
print("Let's chat! (Type 'quit' to end the conversation)\n")

while True:
    # Get what the user wants to say
    user_input = input('👤 You: ')
    
    # Check if the user wants to quit
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("\n👋 Thanks for chatting!")
        break
    
    try:
        # Send the message and all previous history to the AI
        response = chat(
            'llama3.2',
            messages=[*messages, {'role': 'user', 'content': user_input}],
        )
        
        # Show the AI's response
        print(f"\n🤖 AI: {response.message.content}\n")
        
        # Add both the user's message and AI's response to the history
        messages.extend([
            {'role': 'user', 'content': user_input},
            {'role': 'assistant', 'content': response.message.content},
        ])
        
    except Exception as e:
        print(f"\n❌ Oops! Something went wrong: {e}\n")


### How Chat History Works 🤔

When we chat with the AI, we're doing something pretty clever:

1. We keep a list of all messages (both yours and the AI's) in the `messages` list
2. Each message has:
   - A `role`: either 'user' (you) or 'assistant' (the AI)
   - `content`: what was said

3. Every time we send a new message:
   - We include ALL previous messages
   - This helps the AI understand the context
   - The AI can refer back to what was said before

This is why the AI can understand follow-up questions like:
- "Can you explain that more?"
- "Why is that?"
- "What about in winter?"

Try asking some follow-up questions in the chat above to see how the AI uses the conversation history! 🚀


## Practice Time! 🎮

Now it's your turn to try! Here are some fun exercises you can try:

1. **Simple Chat**:
   - Ask the AI to explain your favorite subject in a fun way
   - Ask for a short story about a topic you're interested in
   - Have the AI explain a difficult concept using an analogy

2. **Yes/No Questions**:
   - Is the speed of light faster than the speed of sound?
   - Are dolphins mammals?
   - Can humans breathe underwater naturally?

3. **Multiple Choice**:
   - Create your own quiz about a topic you're studying
   - Ask the AI to choose the best answer
   - Save the conversation to review later

Remember:
- You can adjust the `temperature` to make the AI more creative (higher) or more focused (lower)
- Always save interesting conversations using our `save_chat` function
- Try different AI models if they're available on your computer

Have fun chatting with AI! 🚀